In [42]:
# To add new city, search for metroID using this API call:
# Then change 'region' below to CITY_NAME
# In following cell, you will need metro ID before inserting into SQL Table
import requests

# API token for Songkick (provide your own!)
token = 'XXXXXXXXX'
# ID for metro area
#region = 'San Francisco' #San Francisco metroID=26330
#region = 'Los Angeles'  #Los Angeles metroID=17835
#region = 'Chicago'   #Chicago metroID=9426
region = 'San Diego' #San Diego = 11086
#region = 'Cincinnati' #Cincinnati = 22040
per_page = 50 # Results per page

# 26 venues in Oakland
query_params = { 'apikey': token,
                 'query': region,
                 'per_page': per_page,
                 'page': 1,
         }

# Base URL for event search
#endpoint = 'http://api.songkick.com/api/3.0/metro_areas/'+mid+'/calendar.json?'
# Base URL for venue search
endpoint = 'http://api.songkick.com/api/3.0/search/venues.json?'

# One preliminary call to get the total number of entries:
response = requests.get(endpoint, params=query_params)
total_results = response.json()['resultsPage']['totalEntries']
print "Total number of entries:", total_results
total_calls = int(total_results/per_page) # Need at least this many, plus:
# If 151, or 242, need one more call to grab remainder
if float(total_results)/per_page - int(total_results/per_page) > 0:
    total_calls += 1

# For storing raw venue info, 50 per entry
all_venues_raw = []
for p in range(total_calls):
    # Starts at 0, so +1 for page=1
    query_params['page'] = (p+1)
    response = requests.get( endpoint, params=query_params)
    full_data = response.json()['resultsPage']['results']['venue']
    # Append to our list
    all_venues_raw.append(full_data)
    if p % 20 == 0:
        print "Made %i calls, corresponding to %i venues..." % (p, p*50)

# Sanity check
print "Done!"
print "Found this many pages of venues:", len(all_venues_raw)



Total number of entries: 2620
Made 0 calls, corresponding to 0 venues...
Made 20 calls, corresponding to 1000 venues...
Made 40 calls, corresponding to 2000 venues...
Done!
Found this many pages of venues: 53


In [ ]:
# Sanity checks here, optional cell
print all_venues_raw[4][3]
for page in all_venues_raw:
    for venue in page:
        if venue['metroArea']['id'] != 17835:
            print venue['id'] #venue['metroArea']['id']


In [43]:
#all_venues_raw is list, each entry holds raw information for 50 venues

import pymysql as mdb
con = mdb.connect('localhost', 'root', '', 'scenehash', autocommit=True, charset='utf8', use_unicode=True) 
cur = con.cursor()

"""
cur.execute("DROP TABLE IF EXISTS venues")
cur.execute("CREATE TABLE venues"
            "(pid INT PRIMARY KEY AUTO_INCREMENT, "
            "id INT, "
            "name VARCHAR(150), "
            "city_name VARCHAR(50), "
            "metro_id INT, "
            "latitude VARCHAR(20), "
            "longitude VARCHAR(20), "
            "url VARCHAR(200) ) "
            )
"""
# Following code checks that venue doesn't already exist to avoid multiple entries
for page in all_venues_raw:
    for venue in page:
        city = venue['city']['displayName']
        metroId = venue['metroArea']['id']
        # LA=17835, SF=26330, Chicago=9426, SD=11086, CIN=22040
        if (metroId != 11086):
            continue
        lat = venue['lat']
        lng = venue['lng']
        url = venue['uri']
        venueId = venue['id']
        name = venue['displayName']
        #print venueId
        cur.execute("SELECT * FROM venues WHERE id = %i" % venueId)
        rows = cur.fetchall()
        if not rows: 
            
            #print venueId, city, metroId, lat, lng, url
            sql_str = 'INSERT INTO venues (id, city_name, metro_id, latitude, longitude, url) VALUES (%s, %s, %s, %s, %s, %s)'
            sql_vals = ( venueId, city, metroId, lat, lng, url)
            cur.execute(sql_str, sql_vals)
            print "Inserted:", venueId #, city, metroId, lat, lng, url

